# Bring your own LLMs

Ragas uses langchain under the hood for connecting to LLMs for metrices that require them. This means you can swap out the default LLM we use (`gpt-3.5-turbo-16k`) to use any 100s of API supported out of the box with langchain.

- [Completion LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.llms)
- [Chat based LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chat_models)

This guide will show you how to use another or LLM API for evaluation.

## Evaluating with GPT4

Ragas uses gpt3.5 by default but using gpt4 for evaluation can improve the results so lets use that for the `Faithfulness` metric

To start-off, we initialise the gpt4 `chat_model` from langchain

In [1]:
%pip show ragas

Name: ragas
Version: 0.0.14
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /home/inflaton/miniconda3/envs/chatpdf/lib/python3.11/site-packages
Requires: datasets, langchain, numpy, openai, pydantic, pysbd, sentence-transformers, transformers
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ["OPENAI_API_KEY"] 

True

Now initialise `Faithfulness` with `gpt4`

In [3]:
from langchain.chat_models import ChatOpenAI
from ragas.metrics import Faithfulness, AnswerRelevancy

gpt4 = ChatOpenAI(model_name="gpt-4")
faithfulness_gpt4 = Faithfulness(name="faithfulness", llm=gpt4)
answer_relevancy_gpt4 = AnswerRelevancy(name="answer_relevancy", llm=gpt4)


That's it!

Now lets run the evaluations using the example from [quickstart](../quickstart.ipnb).

In [4]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [5]:
dataset = fiqa_eval["baseline"]
new_ds = dataset.map(
    lambda record: {
        "answer": record["ground_truths"][0],
    },
    batched=False,
)
new_ds

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [6]:
new_ds.to_pandas()

,question,ground_truths,answer,contexts
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,Have the check reissued to the proper payee.Ju...,[Just have the associate sign the back and the...
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...","""I'm afraid the great myth of limited liabilit...",[Set up a meeting with the bank that handles y...
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,You should probably consult an attorney. Howev...,[The time horizon for your 401K/IRA is essenti...
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,Most items used in business have to be depreci...,[You would report it as business income on Sch...
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,Yes. I can by all means start my own company a...,[Yes. I can by all means start my own company ...
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...","""If your sole proprietorship losses exceed all...","[""Short answer, yes. But this is not done thro..."
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,You should have a separate business account. M...,"[""I have checked with Bank of America, and the..."
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...


In [7]:
%%time
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)

result = evaluate(
    new_ds,
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

result

evaluating with [faithfulness]


100%|██████████| 2/2 [07:22<00:00, 221.24s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [01:00<00:00, 30.25s/it]


CPU times: user 659 ms, sys: 56.1 ms, total: 716 ms
Wall time: 8min 23s


{'ragas_score': 0.7649, 'faithfulness': 0.7009, 'answer_relevancy': 0.8417}

In [8]:
%%time
# evaluate

result_gpt4 = evaluate(
    new_ds,
    metrics=[
        faithfulness_gpt4,
        answer_relevancy_gpt4,
    ],
)

result_gpt4

evaluating with [faithfulness]


100%|██████████| 2/2 [27:35<00:00, 827.75s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [02:04<00:00, 62.28s/it]


CPU times: user 609 ms, sys: 64.5 ms, total: 673 ms
Wall time: 29min 40s


{'ragas_score': 0.7457, 'faithfulness': 0.6621, 'answer_relevancy': 0.8535}

In [21]:
result, result_gpt4

({'ragas_score': 0.9147, 'faithfulness': 1.0000, 'answer_relevancy': 0.8428},
 {'ragas_score': 0.9231, 'faithfulness': 1.0000, 'answer_relevancy': 0.8573})

In [10]:
df = result.to_pandas()
best_ds = df[(df.faithfulness > 0.99)]
best_ds

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...,1.0,0.844849
3,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.813215
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.841483
13,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...","I don't have any experience in this, but this ...","[I don't have any experience in this, but this...",1.0,0.857444
14,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.912363
15,Do I need a new EIN since I am hiring employee...,[You don't need to notify the IRS of new membe...,I called the IRS (click here for IRS contact i...,[I called the IRS (click here for IRS contact ...,1.0,0.903124
16,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...,1.0,0.798218
22,Challenged an apparently bogus credit card cha...,[If you tell the collector that the claim isn'...,Yes. For $15 it's not worth anyone's time to ...,[Yes. For $15 it's not worth anyone's time to...,1.0,0.877397
26,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing...",1.0,0.822435
27,Will one’s education loan application be rejec...,[Edit: lazy math The answer to this question d...,A bank can reject a loan if they feel you do n...,[A bank can reject a loan if they feel you do ...,1.0,0.825967


In [11]:
best_ds.index

Index([1, 3, 9, 13, 14, 15, 16, 22, 26, 27, 28], dtype='int64')

In [12]:
df = result_gpt4.to_pandas()
best_ds_gpt4 = df[(df.faithfulness > 0.99)]
best_ds_gpt4

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...,1.0,0.774951
3,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.854237
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,Yes. I can by all means start my own company a...,[Yes. I can by all means start my own company ...,1.0,0.852977
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.870913
10,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...","""For simplicity, let's start by just consideri...","[""For simplicity, let's start by just consider...",1.0,0.873168
14,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.918998
16,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...,1.0,0.798891
18,financial institution wants share member break...,"[""What exactly would the financial institution...","""What exactly would the financial institution ...","[""What exactly would the financial institution...",1.0,0.867838
22,Challenged an apparently bogus credit card cha...,[If you tell the collector that the claim isn'...,Yes. For $15 it's not worth anyone's time to ...,[Yes. For $15 it's not worth anyone's time to...,1.0,0.864591
24,Following an investment guru a good idea?,[I think following the professional money mana...,"""The best answer here is """"maybe, but probably...","[""The best answer here is """"maybe, but probabl...",1.0,0.882338


In [13]:
best_ds_gpt4.index

Index([2, 3, 6, 9, 10, 14, 16, 18, 22, 24, 26, 27], dtype='int64')

In [14]:
baseline_index = list(set(best_ds_gpt4.index).intersection(best_ds.index))
baseline_index

[3, 9, 14, 16, 22, 26, 27]

In [15]:
baseline_ds = new_ds.select(baseline_index)
baseline_ds.to_pandas()

,question,ground_truths,answer,contexts
0,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...","""I'm afraid the great myth of limited liabilit...",[Set up a meeting with the bank that handles y...
1,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...
2,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...
3,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...
4,Challenged an apparently bogus credit card cha...,[Yes. For $15 it's not worth anyone's time to...,Yes. For $15 it's not worth anyone's time to ...,[If you tell the collector that the claim isn'...
5,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing..."
6,Will one’s education loan application be rejec...,[A bank can reject a loan if they feel you do ...,A bank can reject a loan if they feel you do n...,[Edit: lazy math The answer to this question d...


In [16]:
%%time
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)

result = evaluate(
    baseline_ds,
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

result

evaluating with [faithfulness]


100%|██████████| 1/1 [01:20<00:00, 80.20s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:14<00:00, 14.52s/it]


CPU times: user 179 ms, sys: 4.41 ms, total: 183 ms
Wall time: 1min 35s


{'ragas_score': 0.9147, 'faithfulness': 1.0000, 'answer_relevancy': 0.8428}

In [17]:
result.to_pandas()

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.813341
1,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.841869
2,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.912363
3,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...,1.0,0.798029
4,Challenged an apparently bogus credit card cha...,[If you tell the collector that the claim isn'...,Yes. For $15 it's not worth anyone's time to ...,[Yes. For $15 it's not worth anyone's time to...,1.0,0.885483
5,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing...",1.0,0.822660
6,Will one’s education loan application be rejec...,[Edit: lazy math The answer to this question d...,A bank can reject a loan if they feel you do n...,[A bank can reject a loan if they feel you do ...,1.0,0.825963


In [18]:
%%time
# evaluate
from ragas import evaluate

result_gpt4 = evaluate(
    baseline_ds,
    metrics=[
        faithfulness_gpt4,
        answer_relevancy_gpt4,
    ],
)

result_gpt4

evaluating with [faithfulness]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [06:10<00:00, 370.23s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:22<00:00, 22.44s/it]


CPU times: user 207 ms, sys: 18.7 ms, total: 225 ms
Wall time: 6min 33s


{'ragas_score': 0.9231, 'faithfulness': 1.0000, 'answer_relevancy': 0.8573}

In [19]:
result_gpt4.to_pandas()

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.838069
1,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.875386
2,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.922657
3,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...,1.0,0.798246
4,Challenged an apparently bogus credit card cha...,[If you tell the collector that the claim isn'...,Yes. For $15 it's not worth anyone's time to ...,[Yes. For $15 it's not worth anyone's time to...,1.0,0.864317
5,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing...",1.0,0.869685
6,Will one’s education loan application be rejec...,[Edit: lazy math The answer to this question d...,A bank can reject a loan if they feel you do n...,[A bank can reject a loan if they feel you do ...,1.0,0.832500


In [20]:
result, result_gpt4

({'ragas_score': 0.9147, 'faithfulness': 1.0000, 'answer_relevancy': 0.8428},
 {'ragas_score': 0.9231, 'faithfulness': 1.0000, 'answer_relevancy': 0.8573})